# Staging Gold tickers data

#### Script that combines beautyfull soup to extract tickers from a static website and the api service from Alpha vantage to load in data.
#### Data will be written to a pandas data frame and transfered in to Sequel Server Management Service.

### Load in necessary libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import yfinance as yf
from datetime import datetime
import matplotlib as plt
from typing import List
import sqlalchemy
from io import StringIO

### Function to retrieve gold tickers

In [2]:
# Get the concerning gold tickers using https://etfdb.com/etfs/commodity/gold/ as a source
# The source is a static website which we scrape using BeautifulSoup
def get_gold_tickers(source_url: str) -> List:
    """
        This function scrapes the concerning source website using BeautifulSoup 
        and looks for tickers in an expected website structure and returns these 
        as a List object.
        
        parameters: 
            -source_url: str -> source url to scrape the ticker values from  
        
        return: List -> list of gold tickers
    """
    l_tickers = []
    r = requests.get(source_url)
    soup = BeautifulSoup(r.text, 'html.parser')

    etf_source_table = soup.find_all('tr')
    
    # parse source table
    for tr in etf_source_table:
        a_tag = tr.find('a')
        if a_tag:
            ticker = a_tag.text
            if ticker == 'Click Here to Join to ETF Database Pro for 14 Days Free, Export This Data & So Much More':
                break  
            l_tickers.append(ticker) 

    return l_tickers 

### Function to retrieve gold ticker data from Alpha Vantage

In [3]:
def get_gold_ticker_values(tickers: List, api_key: str):
    """
        This function downloads the concerning gold ticker values from Alpha vantage
        and returns this result as a Pandas Dataframe.
        
        parameters: 
            -datatype: CSV format.
            -apikey : API key to connect to Alpha vantage, take notice on only 25 request a day.
            -outputsize: full is nescesary to retrieve maximum history of ticker data, aprox. 20+ years depending on ticker.
            -symbol: ticker symbol according to Alpha vantage.
        
        return:
            -a Pandas DataFrame
    """
    
    df = pd.DataFrame()
    
    params = {
        'datatype' : 'csv',
        'apikey' : api_key,
        'outputsize' : 'full'
    }
    
    for ticker in tickers:
        params['symbol'] = ticker
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
        
        r = requests.get(
            url=url,
            params=params
        )
        
        df_tmp = pd.read_csv(StringIO(r.text))
        df_tmp['ticker'] = ticker
        
        df = pd.concat([df, df_tmp], ignore_index=True)
    
    return df

### Function to write Panda data frame to SQL database

In [7]:
def write_df_to_rds(df: pd.DataFrame, conn, schema, table_name):
    """
        This function writes an incoming Pandas Dataframe to the RDS Database. 
        
        parameters: 
            -df: incoming Pandas Dataframe
            -conn: incoming SQL Alchemy engine object 
            -schema: table schema within the RDS database
            -table_name: destination table name within the RDS database
    """
    
    df.to_sql(
        name=table_name, 
        con=conn, 
        schema=schema,
        if_exists='replace'
    )

In [5]:
# Api key for Alpha Vantage
"""
api_key is only valid for 25 request a day due to ip limit, if necessary to run the script more, change ip.
"""

api_key = 'MZ2H3LL81V48N7OP'

df = get_gold_ticker_values(
    get_gold_tickers('https://etfdb.com/etfs/commodity/gold/'),
    api_key
)


In [8]:
# Define static RDS connection credentials
servername = 'NB-SMET-5CD3102PV6'
dbname = 'syntra_dwh'
conn = sqlalchemy.create_engine(
    'mssql+pyodbc://@' + servername + '/' + dbname + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
) 

write_df_to_rds(
    df,
    conn,
    'staging',
    'stg_alpha_vantage_gold_ticker_figures'
)